In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
ROOT = "/content/drive/MyDrive/threat_project"
os.chdir(ROOT)
print("cwd:", os.getcwd())
print("files:", os.listdir(".")[:50])


Mounted at /content/drive
cwd: /content/drive/MyDrive/threat_project
files: ['port_detector.py', 'phishing_detector.py', 'bruteforce_detector.py', 'master_console.py', 'web_ui.py', 'create_big_dataset.py', 'big_threat_dataset.csv', 'ids_model.pkl', 'blocked_ips.txt', 'detections.json', 'web_ui.log']


In [5]:
!pip install pandas scikit-learn joblib flask pyngrok

In [3]:
import subprocess
proc = subprocess.Popen(["python", "master_console.py"])

In [6]:
# start web_ui.py and expose via ngrok
from pyngrok import ngrok
import subprocess, time, os, signal, sys

# optional: set your ngrok authtoken (uncomment if you have one)
ngrok.set_auth_token("2mEwJxgxSYm4OPapscHO5Vu5PcH_48RgJhqiiDXTY2gGoj5Ds")

# remove any previous tunnel
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
    except Exception:
        pass

# log file for Flask output
FLASK_LOG = "web_ui.log"

# start web_ui.py in background
proc = subprocess.Popen([sys.executable, "web_ui.py"],
                        stdout=open(FLASK_LOG, "w"),
                        stderr=open(FLASK_LOG, "w"))

# give it a moment to start
time.sleep(1.5)

# open ngrok TCP/HTTP tunnel to port 5000 (HTTP)
public_tunnel = ngrok.connect(5000, "http")
public_url = public_tunnel.public_url

print("web_ui.py started in background (pid {}).".format(proc.pid))
print("ngrok public URL:", public_url)
print("Flask logs (last 20 lines):\n")
!sleep 1
!tail -n 20 web_ui.log || true

# keep handles so later you can stop the process / tunnel
_global = globals()
_global["_webui_proc"] = proc
_global["_webui_ngrok_url"] = public_url

web_ui.py started in background (pid 3016).
ngrok public URL: https://ae4588ac9013.ngrok-free.app
Flask logs (last 20 lines):



In [ ]:
# If it says Port 5000 already in use, run this
proc = globals().get("_webui_proc", None)
if proc:
    try:
        proc.terminate()
        print("Terminated stored _webui_proc (pid {})".format(proc.pid))
    except Exception as e:
        print("Could not terminate stored process:", e)
else:
    print("_webui_proc not found in globals")

Terminated stored _webui_proc (pid 3940)


In [ ]:
# If it says Port 5000 already in use, run this
!ss -ltnp | grep ':5000' || echo "port 5000 is free"

In [7]:
# If it shows 500 Internal Server Error, use this to see what really happened
!tail -n 400 web_ui.log || true

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
[2025-10-15 05:34:49,596] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [8]:
# check if the web_ui.py process is alive
import psutil, os, sys
proc = globals().get("_webui_proc", None)
print("Stored _webui_proc:", proc)
if proc:
    pid = proc.pid
    alive = psutil.pid_exists(pid)
    print("web_ui.py PID", pid, "alive:", alive)
else:
    # fallback: search for python processes that look like web_ui
    import subprocess
    !ps aux | grep web_ui.py || true

Stored _webui_proc: <Popen: returncode: None args: ['/usr/bin/python3', 'web_ui.py']>
web_ui.py PID 3016 alive: True


In [ ]:
# If you wanna restart the whole process
# stop old webui
proc = globals().get("_webui_proc", None)
if proc:
    try:
        proc.terminate()
    except:
        pass
# kill by name if necessary
!pkill -f web_ui.py || true
!sleep 1

# remove old log to see fresh output
!rm -f web_ui.log || true

# start web_ui.py
import subprocess, time, sys
p = subprocess.Popen([sys.executable, "web_ui.py"], stdout=open("web_ui.log","w"), stderr=open("web_ui.log","w"))
globals()["_webui_proc"] = p
time.sleep(1.5)
print("Started web_ui.py pid", p.pid)
!tail -n 120 web_ui.log || true

In [ ]:
# Force a test Detection
import json, time, os
p = "detections.json"
logs = json.load(open(p))
logs.append({"tool":"phish","timestamp":time.time(),"src":"1.2.3.4","dst":"-","url":"test-phish-xyz.com","heuristic_score":7})
json.dump(logs, open(p,"w"), indent=2)
print("Wrote test detection. detections.json length:", len(logs))

In [ ]:
import os, time, json, joblib, pandas as pd
from sklearn.tree import DecisionTreeClassifier
MODEL_FILE = os.path.join(ROOT,"ids_model.pkl")
DATA_FILE = os.path.join(ROOT,"big_threat_dataset.csv")
LOG_FILE = os.path.join(ROOT,"detections.json")
BLOCKED = os.path.join(ROOT,"blocked_ips.txt")
last_dataset_len = 0

def load_dataset():
    if not os.path.exists(DATA_FILE):
        return pd.DataFrame(columns=["AttackType","SourceIP","DestIP","Port","URL/Domain","Username","Attempts","HumanDecision","HumanReason"])
    return pd.read_csv(DATA_FILE)

def train_model_if_needed():
    global last_dataset_len
    df = load_dataset()
    if len(df) <= last_dataset_len:
        return
    last_dataset_len = len(df)
    train_df = df[df["HumanDecision"].isin(["Safe","Prevent"])]
    if len(train_df) < 20:
        print("Not enough labeled rows to retrain yet:", len(train_df))
        return
    X = pd.DataFrame()
    X["port_n"] = pd.to_numeric(train_df["Port"].apply(lambda x: str(x).split('-')[0] if str(x).isdigit() else 0), errors="coerce").fillna(0)
    X["attempts_n"] = pd.to_numeric(train_df["Attempts"], errors="coerce").fillna(0)
    X["phish_flag"] = train_df["URL/Domain"].apply(lambda u: 1 if isinstance(u,str) and any(k in u for k in ["secure","login","verify","update","confirm","bank","paypal"]) else 0)
    y = (train_df["HumanDecision"]=="Prevent").astype(int)
    clf = DecisionTreeClassifier(max_depth=6)
    clf.fit(X, y)
    joblib.dump(clf, MODEL_FILE)
    print("Retrained model on", len(train_df), "labeled rows and saved to", MODEL_FILE)

def process_detections():
    if not os.path.exists(LOG_FILE):
        return
    try:
        logs = json.load(open(LOG_FILE,"r"))
    except:
        logs = []
    processed = []
    model = joblib.load(MODEL_FILE) if os.path.exists(MODEL_FILE) else None
    for i, e in enumerate(logs):
        if e.get("tool") == "port":
            port = int(e.get("port",0))
            attempts = 0
            phish_flag = 0
        elif e.get("tool") == "brute":
            port = int(e.get("port",0)) if e.get("port") else 22
            attempts = int(e.get("attempts",0))
            phish_flag = 0
        else:
            port = 0
            attempts = 0
            phish_flag = 1 if e.get("url") and any(k in e.get("url") for k in ["secure","login","verify","update","confirm","bank","paypal"]) else 0
        X = [[port, attempts, phish_flag]]
        if model is not None:
            try:
                pred = model.predict(X)[0]
                if pred == 1:
                    ip = e.get("src","-")
                    with open(BLOCKED,"a") as f:
                        f.write(str(ip)+"\n")
                    print("Auto-Prevented", ip, "from", e.get("tool"))
                    processed.append(i)
                    continue
            except Exception as ex:
                pass
        # if we reach here, unknown or model not certain: leave for human
    # remove processed entries by index
    remaining = [v for idx,v in enumerate(logs) if idx not in processed]
    json.dump(remaining, open(LOG_FILE,"w"), indent=2)

print("Starting continuous ML loop. Press stop to end cell.")
while True:
    train_model_if_needed()
    process_detections()
    time.sleep(20)

Starting continuous ML loop. Press stop to end cell.
Retrained model on 177 labeled rows and saved to /content/drive/MyDrive/threat_project/ids_model.pkl
Auto-Prevented 45.88.232.38 from brute
Auto-Prevented 45.10.98.110 from brute
Auto-Prevented unknown from phish
Auto-Prevented 45.80.78.77 from brute
Auto-Prevented unknown from phish
Auto-Prevented 45.156.248.241 from brute
Auto-Prevented unknown from phish
Auto-Prevented 45.116.217.59 from brute
Auto-Prevented 1.2.3.4 from phish
Auto-Prevented 45.18.169.148 from brute
Auto-Prevented unknown from phish
Auto-Prevented unknown from phish
Auto-Prevented 45.138.71.240 from brute
Auto-Prevented 1.2.3.4 from phish


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local

Auto-Prevented 45.97.98.119 from brute
Auto-Prevented 45.96.42.161 from brute


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local

Auto-Prevented unknown from phish
Auto-Prevented 45.6.24.97 from brute


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local

Auto-Prevented unknown from phish


KeyboardInterrupt: 